In [1]:
%%capture
!pip install "openai>=1" "langchain>=0.0.331rc2" "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken watermark



In [2]:
%%capture
!apt-get update


!sudo apt-get install poppler-utils tesseract-ocr

In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [4]:


import os
import openai
import io
import uuid
import base64
from base64 import b64decode
import numpy as np
from PIL import Image

from unstructured.partition.pdf import partition_pdf

from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from operator import itemgetter

In [5]:

import random
import numpy as np

# Set the seeds
random.seed(42)
np.random.seed(42)

In [6]:
from google.colab import userdata

api_key = userdata.get('OPENAI_KEY')           # <-- change this as per your secret's name
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = OPENAI_KEY
#openai.api_key = os.environ["OPENAI_API_KEY"]



# New Section

In [7]:
# load the pdf file to drive
# split the file to text, table and images
def doc_partition(path,file_name):

  raw_pdf_elements = partition_pdf(
    filename=path + file_name,
    image_output_dir_path=path,
     extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    )

  return raw_pdf_elements
poppler_path = "/content/"
path = "/content/"
file_name = "multi_modal_fin.pdf"
raw_pdf_elements = doc_partition(path,file_name)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [8]:
# appending texts and tables from the pdf file
def data_category(raw_pdf_elements): # we may use decorator here
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
           tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
           texts.append(str(element))
    data_category = [texts,tables]
    return data_category
texts = data_category(raw_pdf_elements)[0]
tables = data_category(raw_pdf_elements)[1]
print(texts)
print(tables)

['ADANI ENTERPRISES LIMITED\n\n176 ANNUAL REPORT 2022-23\n\n(b) Depositories:\n\n1. National Securities Depository Limited (NSDL) Trade World, 4th Floor, Kamala Mills Compound, Senapati Bapat Marg, Lower Parel, Mumbai – 400 013.\n\n2. Central Depository Services (India) Limited (CDSL)\n\n25th Floor, A Wing, Marathon Futurex, Mafatlal Millis Compound, NM Joshi Marg, Lower Parel (E), Mumbai - 400013\n\nAnnual Listing fee of BSE & NSE and Annual Custody / Issuer fee of NSDL & CDSL for FY 2023-24 are paid to respective agencies.\n\nI. Market Price Data: High, Low during each month in Financial Year 2022-23.\n\nMonthly share price movement during the year 2022-23 at BSE & NSE:', 'Volume\n\n[Source: This information is compiled from the data available from the websites of BSE and NSE]\n\nJ. Performance in comparison to broad-based indices such as BSE Sensex.\n\nBSE Sensex  AEL Share Price  64,000.00  4500.00  62,000.00  4000.00  X  60,000.00  3500.00  e  c  i  E  S  N  E  S     E  S  B  58,0

In [9]:
# function to take tables and text and then summarize tables only,
def tables_summarize(data_category):
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
                    Give a concise summary of the table or text. Table or text chunk: {element} """

    prompt = ChatPromptTemplate.from_template(prompt_text)
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
    table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
    #text_summaries =  summarize_chain.batch(data_category[0], {"max_concurrency": 5})# no need to summarize

    return table_summaries
table_summaries = tables_summarize(data_category)
text_summaries = texts

print(table_summaries)

<ipython-input-9-86a6771886a2>:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0, model="gpt-4")


['The table provides a monthly breakdown of the high and low share prices on the BSE and NSE from April 2022 to March 2023. The highest share price on the BSE was in December 2022 at 4,189.55, while the lowest was in February 2023 at 1,017.10. The highest volume of shares traded was in March 2023 with 5,52,47,455 shares. On the NSE, the highest share price was also in December 2022 at 4,190.00, and the lowest was in February 2023 at 1,017.45. The highest volume of shares traded on the NSE was in March 2023 with 22,80,29,587 shares.']


In [10]:
def encode_image(image_path):
    ''' Getting the base64 string '''
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def image_captioning(img_base64,prompt):
    ''' Image summary '''
   # chat = ChatOpenAI(model="gpt-4-vision-preview",
    chat = ChatOpenAI(model="gpt-4o-mini",
                       max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text":prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_base64}"
                        },
                    },
                ]
            )
        ]
    )
    return msg.content

In [11]:
import time
# Store base64 encoded images
path = "/content/figures"
img_base64_list = []

# Store image summaries
image_summaries = []

# Prompt
prompt = "Describe the image in detail. Be specific about graphs, such as bar plots."

# Read images, encode to base64 strings
for img_file in sorted(os.listdir(path)):
    if img_file.endswith('.jpg'):
        img_path = os.path.join(path, img_file)
        base64_image = encode_image(img_path)
        img_base64_list.append(base64_image)
        #image_summaries.append(image_captioning(base64_image,prompt))
        img_cap = image_captioning(base64_image,prompt)
        time.sleep(5)
        image_summaries.append(img_cap)

print(image_summaries)


['The image presents a line graph comparing the BSE Sensex and AEL Share Price over a span of several months from April 2022 to March 2023.\n\n### Graph Details:\n\n- **X-Axis (Months)**: The horizontal axis is labeled "Months" and includes the months from April 2022 to March 2023. Each month is evenly spaced along this axis.\n\n- **Y-Axis (BSE Sensex)**: The left vertical axis is labeled "BSE SENSEX," with values ranging from 44,000 to 64,000, marked in increments of 2,000.\n\n- **Y-Axis (AEL Share Price)**: The right vertical axis is labeled "AEL Share Price," with values ranging from 0 to 4,500, marked in increments of 500.\n\n### Data Representation:\n\n- **BSE Sensex**:\n  - Represented by a gray line, it shows a fluctuating trend over the months.\n  - The line generally trends upwards from April 2022 to a peak around September 2022, after which it shows some decline towards the end of the year, with a notable drop in January 2023 before recovering slightly.\n\n- **AEL Share Price

In [12]:
def split_image_text_types(docs):
    ''' Split base64-encoded images and texts '''
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {
        "images": b64,
        "texts": text
    }

In [ ]:
# Create a vectore base and store text,table,images and their index

In [13]:
# Add raw docs and doc summaries to Multi Vector Retriever.
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag",
                     embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    cache=False,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

if summary_img:
    retriever.vectorstore.add_documents(summary_img)
    retriever.docstore.mset(list(zip(img_ids, img_base64_list)))
else:
    print("No image summaries to add to the vectorstore.")

<ipython-input-13-c1d23303cdc8>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function=OpenAIEmbeddings())
<ipython-input-13-c1d23303cdc8>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="multi_modal_rag",


In [14]:
# Add raw docs and doc summaries to Multi Vector Retriever.
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag",
                     embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    cache=False,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

if summary_img:
    retriever.vectorstore.add_documents(summary_img)
    retriever.docstore.mset(list(zip(img_ids, img_base64_list)))
else:
    print("No image summaries to add to the vectorstore.")

from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def prompt_func(dict,seed=None):
    format_texts = "\n".join(dict["context"]["texts"])
    # Check if images are present before accessing them
    if dict["context"]["images"]:
        image_url = f"data:image/jpeg;base64,{dict['context']['images'][0]}"
    else:
        image_url = None  # Or provide a default image URL if needed
    return [
        HumanMessage(
            content=[
                {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
Question: {dict["question"]}

Text and tables:
{format_texts}
"""},
 #               {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{dict['context']['images'][0]}"}},
 #           ]
                 *([{"type": "image_url", "image_url": {"url": image_url}}] if image_url else []),
            ]
        )
    ]

def create_chain(seed=None):  # Function to create chain with seed
    # Set seeds if provided
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    model = ChatOpenAI(temperature=0, model="gpt-4o-mini", seed=seed)

    # RAG pipeline
    chain = (
        {"context": retriever | RunnableLambda(split_image_text_types), "question": RunnablePassthrough()}
        | RunnableLambda(lambda x: prompt_func(x, seed=seed))  # Pass seed to prompt_func
        | model
        | StrOutputParser()
    )
    return chain

# Create chain with a specific seed
chain = create_chain(seed=12345)

chain.invoke(
    "what is BSE high in April 2022 ?"
)


                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.


'The BSE high in April 2022 was 2,420.00.'

In [15]:
chain.invoke(
    "What is BSE sensex in April 2022 include chart?"
)

'In April 2022, the BSE Sensex was approximately 58,000. The chart shows the trend of the BSE Sensex and AEL Share Price over the months, with the Sensex represented by a gray line.'

In [16]:
chain.invoke(
    "What is BSE sensex in April 2022 include chart?"
)

'In April 2022, the BSE Sensex was approximately 58,000. The chart shows the trend of the BSE Sensex and AEL Share Price over the months, with the Sensex fluctuating around this value during that period.'

In [17]:
chain.invoke(
    "what is mean and average top 2 highest stock prices include chart?"
)

'To find the mean and average of the top 2 highest stock prices from the provided data:\n\n1. **Identify the highest stock prices:**\n   - The highest stock price in December 2022: **4,189.55**\n   - The second highest stock price in November 2022: **4,098.10**\n\n2. **Calculate the mean and average:**\n   - **Mean = (Highest + Second Highest) / 2**\n   - Mean = (4,189.55 + 4,098.10) / 2 = 4,143.83\n\nThus, the mean of the top 2 highest stock prices is **4,143.83**. \n\nThe average is the same as the mean in this context, so the average is also **4,143.83**.'

In [ ]:
chain.invoke(
    "what is AEL share price Feb 23 use only data from context ?"
)

'The AEL share price for February 2023 is 1,500.00.'

In [ ]:
chain.invoke(
    "what is INFOSYS share price Feb 23 use only data from context?"
)

'The share price of INFOSYS on February 23, 2023, is not provided in the context. The data available pertains to Adani Enterprises Limited and does not include any information about INFOSYS.'

In [ ]:
chain.invoke(
    "what are Registrar and Share Transfer Agents ?"
)

'Registrar and Share Transfer Agents (R&T Agents) are entities appointed by a company to manage the administrative tasks related to the transfer of shares and maintenance of shareholder records. In the context provided, M/s. Link Intime India Private Limited is the appointed R&T Agent for the company, handling both physical and dematerialized (demat) shares. Their responsibilities include processing share transfers, managing shareholder queries regarding transfers, dividends, and changes of address, and ensuring compliance with regulatory requirements related to share transfers. They play a crucial role in facilitating smooth transactions and communication between the company and its shareholders.'

In [ ]:
chain.invoke(
    "Extract details of March 2023 ?"
)

In [ ]:
chain.invoke(
    "Extract details of March 2023 use image ?"
)